In [1]:
using CairoMakie
using Unitful
using Colors
import PhysicalConstants.CODATA2018: h, k_B, c_0

In [2]:
# Set some plot defaults
custom_theme = Theme(
    Axis = (
        xgridvisible = false,  # Set grid visibility to false
        ygridvisible = false,
    ),
)
Makie.set_theme!(custom_theme)

In [3]:
i_units = u"kW / (m^2 * nm * sr)"

function blackbody_λ(λ, temp) 
    # your code here
end

blackbody_λ (generic function with 1 method)